In [2]:
# 라이브러리 import

from langchain_openai import ChatOpenAI
import dotenv
import os

dotenv.load_dotenv()

True

In [ ]:
# LLM Load

llm = ChatOpenAI(model_name="gpt-4o-mini",
                 temperature=0.,
                 )              


In [ ]:
result = llm.invoke("한국의 수도는 어디인가요?")
print(result)

content='한국의 수도는 서울입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'finish_reason': 'stop', 'logprobs': None} id='run-3e0c9757-426a-4faf-998e-c1d2a3d3725c-0' usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [5]:
result = llm.invoke("""
당신은 한식 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


된장찌개는 무슨 음식인가요?
                    """)

print(result.content)

된장찌개는 한국의 전통적인 찌개 중 하나로, 주재료인 된장을 기본으로 하여 다양한 재료를 넣어 끓인 국물 요리입니다. 된장은 발효된 콩으로 만든 한국의 대표적인 장류로, 깊고 풍부한 맛을 제공합니다.

된장찌개는 보통 두부, 호박, 양파, 버섯, 감자, 청양고추 등 다양한 채소와 함께 조리되며, 때로는 고기(주로 돼지고기나 소고기)도 추가하여 더욱 풍미를 더할 수 있습니다. 이 찌개는 구수한 맛과 함께 건강에도 좋은 영양소가 풍부하여 많은 사람들이 즐겨 먹습니다.

된장찌개는 밥과 함께 먹는 것이 일반적이며, 한국의 가정식에서 자주 등장하는 메뉴입니다. 각 가정마다 레시피가 조금씩 다를 수 있어, 각자의 개성을 담아 요리할 수 있는 매력이 있습니다. 고객분들에게는 이 찌개의 역사와 재료의 조화, 그리고 각 재료가 주는 맛의 변화를 설명해드리면 더욱 흥미롭게 느끼실 수 있을 것입니다.


In [6]:
prompt = \
"""
당신은 한식 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?
"""

result = llm.invoke(prompt)

print(result.content)

저는 한식 요리사로, 25년의 경력을 가지고 있는 전문가입니다. 현재 호텔에서 근무하며, 고객을 위해 정성껏 요리를 준비하고 있습니다. 요리를 통해 한국의 전통과 문화를 전달하고, 고객분들에게 요리에 대한 세심한 설명을 해드리는 것을 좋아합니다. 제 요리가 고객에게 특별한 경험이 되기를 바라며, 항상 최선을 다하고 있습니다.


In [7]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

In [ ]:
# 프롬프트 템플릿릿

prompt = PromptTemplate.from_template(
"""
당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?
""")

In [9]:
print(prompt.format(cuisine="일식"))


당신은 일식 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. 언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.


당신은 누구인가요?



In [10]:
chain = prompt | llm

In [11]:
print(chain.invoke({"cuisine":"일식"}).content)

저는 25년 경력의 일식 요리사로, 현재 호텔에서 근무하고 있습니다. 고객을 위해 정성껏 요리를 만들고, 각 요리에 담긴 의미와 재료에 대한 설명을 세심하게 해드리는 것을 좋아합니다. 요리를 통해 고객에게 특별한 경험을 선사하는 것이 제 사명이라고 생각합니다. 일본의 전통과 현대적인 감각을 조화롭게 담아내는 요리를 통해, 고객들이 만족할 수 있도록 최선을 다하고 있습니다.


In [12]:
print(chain.invoke({"cuisine":"중식"}).content)

저는 중식 요리사로, 25년의 경력을 가지고 있는 전문가입니다. 현재 호텔에서 근무하며, 고객을 위해 정성껏 요리를 준비하고 있습니다. 요리를 통해 고객에게 즐거움을 주는 것을 가장 중요하게 생각하며, 각 요리에 대한 세심한 설명을 통해 그들의 경험을 더욱 풍부하게 만들어 드리는 것을 좋아합니다. 제 요리는 전통적인 중식의 맛을 살리면서도 현대적인 감각을 더해 고객들에게 특별한 경험을 선사하고자 노력하고 있습니다.


In [13]:
# PromptTemplate을 이용한 질문 변수화

prompt2 = PromptTemplate.from_template("""

당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.

{question}

"""
                                       )

In [14]:
print(prompt2.template)



당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. 언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.

{question}




In [15]:
chain2 = prompt2|llm

print(chain2.invoke({"cuisine":"인도",
			         "question":"가장 유명한 음식이 무엇인가요?"}).content)

인도 요리에서 가장 유명한 음식 중 하나는 "버터 치킨"입니다. 이 요리는 부드러운 치킨 조각을 크리미한 토마토 소스와 버터로 조리하여 만든 요리로, 향신료의 조화가 일품입니다. 또한, "비리야니"라는 향신료가 풍부한 쌀 요리도 매우 인기 있습니다. 비리야니는 고기나 채소와 함께 조리되어 풍미가 가득한 한 끼 식사로 사랑받고 있습니다.

그 외에도 "사모사", "팔락 파니르", "다르지"와 같은 다양한 요리들이 있으며, 각 지역마다 특색 있는 요리가 많습니다. 고객분들에게 요리의 유래와 재료에 대해 설명해드리는 것도 큰 즐거움입니다. 어떤 요리에 대해 더 알고 싶으신가요?


In [16]:
# ChatPromptTemplate을 이용한 질문 변수화

prompt2 = ChatPromptTemplate.from_messages([
("system", "당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. \
언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다."),
("human", "{question}")
])

In [17]:
print(prompt2.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['cuisine'], input_types={}, partial_variables={}, template='당신은 {cuisine} 요리사입니다. 현재 호텔에서 근무하고 있으며 경력은 25년입니다. 언제나 고객을 위해 정성껏 요리를 만들고 고객분들에게 세심하게 설명해주는 것을 좋아합니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]


In [18]:
chain2 = prompt2|llm

print(chain2.invoke({"cuisine":"인도",
			         "question":"가장 유명한 음식이 무엇인가요?"}).content)

인도 요리에서 가장 유명한 음식 중 하나는 "버터 치킨"입니다. 이 요리는 부드러운 치킨 조각을 크리미한 토마토 소스와 버터로 조리하여 만든 요리로, 향신료의 조화가 일품입니다. 또한, "비리야니"라는 향신료가 풍부한 쌀 요리도 매우 인기 있습니다. 비리야니는 고기나 채소와 함께 조리되며, 각 지역마다 독특한 스타일이 있습니다.

그 외에도 "사모사", "팔락 파니르", "다르"와 같은 다양한 요리들이 인도 전역에서 사랑받고 있습니다. 각 요리는 지역의 특성과 문화에 따라 다르게 조리되며, 그 맛과 향은 정말 매력적입니다. 고객님께서 어떤 요리를 원하시는지에 따라 추천해드릴 수 있습니다!


In [19]:
print(chain2.invoke({"cuisine":"이탈리아",
			         "question":"가장 유명한 음식이 무엇인가요?"}).content)

이탈리아 요리에서 가장 유명한 음식 중 하나는 피자입니다. 특히 나폴리 스타일의 마르게리타 피자가 세계적으로 유명하죠. 신선한 토마토 소스, 모짜렐라 치즈, 바질 잎이 조화를 이루는 이 피자는 이탈리아의 전통적인 맛을 잘 보여줍니다.

또한, 파스타도 이탈리아 요리의 상징적인 음식입니다. 스파게티, 라자냐, 리가토니 등 다양한 종류가 있으며, 각 지역마다 특색 있는 소스와 조리법이 있습니다. 예를 들어, 볼로네제 소스가 들어간 라자냐는 이탈리아 북부의 대표적인 요리입니다.

이 외에도 리조또, 티라미수, 아란치니 등 다양한 이탈리아 음식들이 세계적으로 사랑받고 있습니다. 각 요리마다 그 지역의 문화와 전통이 담겨 있어, 맛뿐만 아니라 이야기도 함께 즐길 수 있습니다.


In [20]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import WikipediaRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

In [21]:
wikipedia_retriever = WikipediaRetriever(top_k_results=1, lang="ko")

In [22]:
docs = wikipedia_retriever.invoke("돈까스")

In [23]:
docs

[Document(metadata={'title': '돈가스', 'summary': '돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.\n\n', 'source': 'https://ko.wikipedia.org/wiki/%EB%8F%88%EA%B0%80%EC%8A%A4'}, page_content='돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.\n\n\n== 어원 ==\n돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다. \n1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에

In [24]:
len(docs)

1

In [25]:
type(docs)

list

In [26]:
type(docs[0])

langchain_core.documents.base.Document

In [27]:
doc_meta = docs[0].metadata
doc_meta

{'title': '돈가스',
 'summary': '돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.\n\n',
 'source': 'https://ko.wikipedia.org/wiki/%EB%8F%88%EA%B0%80%EC%8A%A4'}

In [28]:
doc = docs[0].page_content

print(doc)

돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.


== 어원 ==
돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다. 
1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에서 포크가쓰레쓰를 개발한 후 돈가스(일본어: 豚かつ 돈카쓰[*])라고 이름 붙였다. 이후 여러 요리책에서는 ‘포크(pork) 가쓰레쓰’, ‘돼지고기 가쓰레쓰’, 돈가스 등으로 혼용되어 불리다가, 1959년 이후에 돈가스라는 이름과 함께 정착해 현재에 이른다.
한국에서는 돈가스 말고도 돈까스, 돈까쓰, 돈카츠 등 여러 이명이 존재한다. 1995년 문화체육부는 '일본어투 생활용어 순화 자료'를 토대로 돈가스를 '돼지고기 너비 튀김'으로 순화할 것을 권장하기도 했다.


== 역사 ==


=== 튀김 문화 ===
불교의 영향이 강했던 7세기 후반 덴무 천황의 육식금지령 이후 1천년 넘게 고기를 먹지 않았다. 에도막부의 도쿠가와 츠나요시가  1687년 ‘동물보호법령’을 선포하여 동물의 살생을 법으로 제한함에 따라 육고기를 구하는 것이 사실상 불

In [29]:
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    separators=['\n\n','\n']
)

In [30]:
split_documents = splitter.create_documents([doc])

In [31]:
split_documents

[Document(metadata={}, page_content='돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.'),
 Document(metadata={}, page_content='== 어원 =='),
 Document(metadata={}, page_content='\n돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다. '),
 Document(metadata={}, page_content='\n1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에서 포크가쓰레쓰를 개발한 후 돈가스(일본어: 豚かつ 돈카쓰[*])라고 이름 붙였다. 이후 여러 요리책에서는 ‘포크(pork) 가쓰레쓰’, ‘돼지고기 가쓰레쓰’, 돈가스 등으로 혼용되어 불리다가, 1959년 이후에 돈가스라는 이름과 함께 정착해 현재에 이른다.'),
 Document(metadata={}, page_content="한국에서는 돈가스 말고도 돈까스, 돈까쓰, 돈카츠 등 여러 이명이 존재한다. 1995년 문화체육부는 '일본어투 생활용어 순화 자료'를

In [32]:
vector_store = FAISS.from_documents(embedding=OpenAIEmbeddings(), documents=split_documents)

In [33]:
retriever = vector_store.as_retriever()

In [34]:
result = retriever.invoke("돈까스는 어느 곳에서 유래됐나요?")

In [35]:
result

[Document(id='9457d469-40e5-4ee0-ac3c-dd928075e38f', metadata={}, page_content="한국에서는 돈가스 말고도 돈까스, 돈까쓰, 돈카츠 등 여러 이명이 존재한다. 1995년 문화체육부는 '일본어투 생활용어 순화 자료'를 토대로 돈가스를 '돼지고기 너비 튀김'으로 순화할 것을 권장하기도 했다."),
 Document(id='1a38cafc-979e-4a4a-ac9d-62b97d8b3a45', metadata={}, page_content='=== 돈가스의 기원 ==='),
 Document(id='3a81838b-8c7c-465d-a510-d3b8c409b4a7', metadata={}, page_content="현대에는, 돈까스만을 전문적으로 요리하는 전문점이 세계적으로 생기고 있다. 세계적으로 유명한 돈까스 전문점으로는, 한국에서 미슐랭 스타를 받은 '남궁현우'씨의 '카츠까스'가 가장 유명하다고 알려져 있다."),
 Document(id='84cfeb45-8f1e-414c-a387-1ad7aaff1c13', metadata={}, page_content='\n돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다. ')]

In [36]:
result = retriever.invoke("궁내청 요리사의 이름은?")

In [37]:
result

[Document(id='bd8bec67-e7b4-402d-84b9-4e6b461e3f0e', metadata={}, page_content='== 파생 요리 ==\n\n\n=== 가츠동 ==='),
 Document(id='5684d633-1a57-4bc5-b6be-245742db333b', metadata={}, page_content='=== 꼬치돈가스 ===\n\n칸사이 지방에서 팔리는 꼬치고기 튀김 요리이다. 돼지고기, 새우, 은어, 양파 등의 요리를 조금씩 곶아 튀겨 먹는 음식이다. 터키의 시시케밥과는 달리 꼬치에 꽂은 고기에 빵가루를 입혀 튀기는 일본만의 요리이다.'),
 Document(id='3a81838b-8c7c-465d-a510-d3b8c409b4a7', metadata={}, page_content="현대에는, 돈까스만을 전문적으로 요리하는 전문점이 세계적으로 생기고 있다. 세계적으로 유명한 돈까스 전문점으로는, 한국에서 미슐랭 스타를 받은 '남궁현우'씨의 '카츠까스'가 가장 유명하다고 알려져 있다."),
 Document(id='f5957242-94d0-4e5a-af97-6291f9c2ad4e', metadata={}, page_content='\n1929년 전직 궁내청 요리사 시마다 신지로(島田信二郎)는 우에노시에 있는 자신의 요리집 폰치켄에서, 2~3cm 두께로 두툼한 돼지고기를 튀긴 돈가스를 팔기 시작했다. 나이프와 포크를 사용하지 않고도 먹도록 칼로 미리 썰어 놓았으므로, 종래의 일식처럼 젓가락으로 먹을 수 있었다.')]

In [52]:
vector_store.docstore._dict

{'e10e22db-676a-49e7-81a2-8530cbd8d9db': Document(id='e10e22db-676a-49e7-81a2-8530cbd8d9db', metadata={}, page_content='돈가스(일본어: 豚カツ (とんかつ) 톤카츠[*], 영어: Tonkatsu 또는 pork cutlet)는 이탈리아의 코톨레타(cotoletta)를 원형으로 하는 서양의 커틀릿에서 유래한 일본 요리이며, 돼지 등심을 2~3 cm 두께로 넓적하게 썰어 빵가루를 묻힌 후, 기름에 튀겨 일본식 우스터 소스(영어: Worcestershire sauce), 밥, 야채 샐러드(주로 양배추 채)를 곁들여 먹는다. 쇠고기를 넣어 튀긴 것은 규카츠(비프 커틀릿, 영어: beef cutlet)라고 한다.'),
 'ec59ae79-d511-4d6f-aaac-72c35ece781c': Document(id='ec59ae79-d511-4d6f-aaac-72c35ece781c', metadata={}, page_content='== 어원 =='),
 '84cfeb45-8f1e-414c-a387-1ad7aaff1c13': Document(id='84cfeb45-8f1e-414c-a387-1ad7aaff1c13', metadata={}, page_content='\n돈가스의 어원은 영국의 커틀릿에서 유래한다. 1872년 가나가키 로분이 저서 “서양요리통”에 ‘포크 커틀릿’을 소개한 이래 1895년 기타 겐지로(木田元次郞)가 운영하던 도쿄 긴자의 양식집 렌가테이(연와정)에서 양배추 채를 곁들인 돈가스의 전신을 ‘돼지고기 커틀릿’, 즉 포크가쓰레쓰(ポークカツレツ)라는 이름으로 팔기 시작하였다 이후 20세기 초 도시를 중심으로 양식 포장마차(洋食屋台)가 생기면서 가쓰레쓰 요리가 정착하기 시작했다. '),
 '8742a92e-8e70-4c0e-8c7f-bff88cd2997b': Document(id='8742a92e-8e70-4c0e-8c7f-bff88cd2997b', metadata={